In [1]:
# Required Python packages
from datetime import date, datetime, timedelta
import calendar
import pandas as pd
import numpy as np
import itertools
import copy
from numpy.linalg import multi_dot
from scipy.stats import norm
from scipy.stats import bernoulli
from scipy.optimize import fmin_slsqp as min
from scipy.optimize import brentq
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from statsmodels.api import OLS

from sklearn.linear_model import LassoCV
from yellowbrick.datasets import load_concrete
from yellowbrick.regressor import AlphaSelection

from scipy.interpolate import CubicSpline

import matplotlib.pyplot as plt
import math
import pickle

#Import Date opertative user defined functions
from ipynb.fs.full.user_defined_vik_functions import get_all_monthly_option_expiries, \
                                                     find_last_thurs_date_of_month, \
                                                     prev_workday_if_holiday, find_wkly_expries,\
                                                     date_of_prev_thurs

# Import dataframe naming functions 
from ipynb.fs.full.user_defined_vik_functions import get_mthly_df_name_from_expiry

#Import data loading functions
from ipynb.fs.full.user_defined_vik_functions import load_all_mthly_data

#import traded options parameters and info
from ipynb.fs.full.user_defined_vik_functions import generate_weekly_strikes, generate_mthly_strikes

#Import pricing functions
from ipynb.fs.full.user_defined_vik_functions import BSM_call_vec_with_div, BSM_put_vec_with_div, call_delta_with_spot_and_div, call_delta_with_spot, put_delta_with_spot, put_delta_with_spot_and_div 

#Import implied volatility processing functions
from ipynb.fs.full.user_defined_vik_functions import generate_weekly_iv, gen_interpolated_iv

#Import Risk-free interest rate generation functions
from ipynb.fs.full.user_defined_vik_functions import generate_weekly_ir

#Import Stochatic process simulation functions
from ipynb.fs.full.user_defined_vik_functions import gbm1W_simulation, generate_covariance_from_correlation

# Import risk free interest rate function
from ipynb.fs.full.user_defined_vik_functions import get_risk_free_rate_from_exact_date


In [56]:
# Stock Index of Interest
# stock_ident = "BANKNIFTY"
stock_ident = "NIFTY"

#Static hedging performed at different moneyness regions 
#i.e. moneyness is used to select the option with nearest moneyness match
# ATM - At the Money, ITM - In the money, OTM - Out of the Money
prod_moneyness = "ATM"

#Product type to hedge: either "CE" or "PE"
prod_type = "CE"
# prod_type = "PE"

call_strikes_list = [1000, 1200, 1400]
put_strikes_list  = [600, 800, 1000]

# The scope of this code is to hedge one option 
# and scope will be extended to a portfolio
no_of_assets = 1
cor_mat = [[1]]

#Path to refer data
source_path = "/home/jupyter-partha/Vikranth - Chapter 2/"
input_sub_path = "Input Data/mkt_data_covid_region/"
output_sub_path = "Output Data/"
input_data_path = source_path + input_sub_path
output_data_path = source_path + output_sub_path
out_PnL_path = source_path + "doc_outputs/PnL_attribution/"


# Periods of interest will be a dictionary
#Key is the year, value is a list of months 1-12, 1- Jan, 2 - Feb,...12 - Dec
# For E.g., periods_of_interest = {2020: [3], 2019: [11, 12]}
periods_of_interest = {2019: [8, 9, 10, 11, 12], 2020: [1, 2, 3, 4, 5, 6, 7]}


#List of holidays
holidays_list = [date(2019, 3, 4), date(2019, 3, 21),\
                 date(2019, 4, 17), date(2019, 4, 19), date(2019, 4, 29),\
                 date(2019, 5, 1),\
                 date(2019, 6, 5),\
                 date(2019, 8, 12), date(2019, 8, 15),\
                 date(2019, 9, 2), date(2019, 9, 10), \
                 date(2019, 10, 2), date(2019, 10, 8), date(2019, 10, 21), date(2019, 10, 28), \
                 date(2019, 11, 12), \
                 date(2019, 12, 25), \
                 date(2020, 2, 21), \
                 date(2020, 3, 10), \
                 date(2020, 4, 2), date(2020, 4, 6), date(2020, 4, 10), date(2020,4, 14), \
                 date(2020, 5, 1), date(2020, 5, 25), \
                 date(2020, 10, 2), date(2020, 11, 16), date(2020, 11, 30), date(2020, 12, 25)]

#Simulation Parameters
no_of_paths = 5000

#Number of options
no_opt=1

def error_func(iv, mkt_price, s_t, k, r, r_f, dt, option_type):
    if (option_type == "CE"):
        model_price = BSM_call_vec_with_div(s_t, k, r, r_f, iv, dt)
    else:
        model_price = BSM_put_vec_with_div(s_t, k, r, r_f, iv, dt)
    error = model_price - mkt_price
    return error


def find_vega(s_t, k, r, r_f, vol, dt):
    d_1_c = (1 / (vol * (dt ** 0.5))) 
    d1_log =  (np.log((s_t)/(k)) + ((r + (vol ** 2) / 2) * dt))
    d_1 = d_1_c * d1_log
    d_2 = d_1 - vol * (dt ** 0.5)
    q = r_f - r
    return (norm.pdf(d_2) * k * np.exp(-r_f * dt)) * np.sqrt(dt)

# def find_vega_test(s_t, k, r, r_f, vol, dt):
#     d_1_c = (1 / (vol * (dt ** 0.5))) 
#     d1_log =  (np.log((s_t)/(k)) + ((r + (vol ** 2) / 2) * dt))
#     d_1 = d_1_c * d1_log
#     d_2 = d_1 - vol * (dt ** 0.5)
#     q = r_f - r
#     return (norm.pdf(d_1) * s_t * np.exp(-q * dt)) * np.sqrt(dt)

def find_gamma(s_t, k, r, r_f, vol, dt):
    vega = find_vega(s_t, k, r, r_f, vol, dt)
    gamma = vega / ((s_t**2) * vol * dt)
    return gamma

# def find_gamma_test(s_t, k, r, r_f, vol, dt):
#     d_1_c = (1 / (vol * (dt ** 0.5))) 
#     d1_log =  (np.log((s_t)/(k)) + ((r + (vol ** 2) / 2) * dt))
#     d_1 = d_1_c * d1_log
#     d_2 = d_1 - vol * (dt ** 0.5)
#     q = r_f - r
#     gamma = np.exp(-q*dt) * (norm.pdf(d_1)) / (vol * s_t * np.sqrt(dt))
#     return gamma

def find_vanna(s_t, k, r, r_f, vol, dt):
    d_1_c = (1 / (vol * (dt ** 0.5))) 
    d1_log =  (np.log((s_t)/(k)) + ((r + (vol ** 2) / 2) * dt))
    d_1 = d_1_c * d1_log
    d_2 = d_1 - vol * (dt ** 0.5)
    q = r_f - r
    vega = find_vega(s_t, k, r, r_f, vol, dt)
    vanna = (vega / s_t) * (1 - ((d_1)/ (vol * np.sqrt(dt))))
    return vanna

def find_volga(s_t, k, r, r_f, vol, dt):
    d_1_c = (1 / (vol * (dt ** 0.5))) 
    d1_log =  (np.log((s_t)/(k)) + ((r + (vol ** 2) / 2) * dt))
    d_1 = d_1_c * d1_log
    d_2 = d_1 - vol * (dt ** 0.5)
    q = r_f - r
    vega = find_vega(s_t, k, r, r_f, vol, dt)
    volga = vega * (d_1 * d_2) / vol
    return volga

def find_speed(s_t, k, r, r_f, vol, dt):
    d_1_c = (1 / (vol * (dt ** 0.5))) 
    d1_log =  (np.log((s_t)/(k)) + ((r + (vol ** 2) / 2) * dt))
    d_1 = d_1_c * d1_log
    d_2 = d_1 - vol * (dt ** 0.5)
    q = r_f - r
    gamma = find_gamma(s_t, k, r, r_f, vol, dt)
    speed = - (gamma / s_t) * ((d_1 / (vol * np.sqrt(dt))) + 1)
    return speed


def find_theta_Call(s_t, k, r, r_f, vol, dt):
    d_1_c = (1 / (vol * (dt ** 0.5))) 
    d1_log =  (np.log((s_t)/(k)) + ((r + (vol ** 2) / 2) * dt))
    d_1 = d_1_c * d1_log
    d_2 = d_1 - vol * (dt ** 0.5)
    q = r_f - r
    term1 = - (np.exp(-q*dt) * s_t * norm.pdf(d_1) * vol) / (2 * np.sqrt(dt))
    term2 = - r_f * k * np.exp(-r_f*dt) * norm.cdf(d_2)
    term3 = q * s_t * np.exp(-q*dt) * norm.cdf(d_1)
    theta = term1 + term2 + term3
    return theta

def find_theta_Put(s_t, k, r, r_f, vol, dt):
    d_1_c = (1 / (vol * (dt ** 0.5))) 
    d1_log =  (np.log((s_t)/(k)) + ((r + (vol ** 2) / 2) * dt))
    d_1 = d_1_c * d1_log
    d_2 = d_1 - vol * (dt ** 0.5)
    q = r_f - r
    term1 = - (np.exp(-q*dt) * s_t * norm.pdf(d_1) * vol) / (2 * np.sqrt(dt))
    term2 =  r_f * k * np.exp(-r_f*dt) * norm.cdf(-d_2)
    term3 = -q * s_t * np.exp(-q*dt) * norm.cdf(-d_1)
    theta = term1 + term2 + term3
    return theta


def find_veta(s_t, k, r, r_f, vol, dt):
    d_1_c = (1 / (vol * (dt ** 0.5))) 
    d1_log =  (np.log((s_t)/(k)) + ((r + (vol ** 2) / 2) * dt))
    d_1 = d_1_c * d1_log
    d_2 = d_1 - vol * (dt ** 0.5)
    q = r_f - r
    veta = -s_t * np.exp(-q*dt) * norm.pdf(d_1) * np.sqrt(dt) * \
    (q + ((r_f-q) * d_1) / (vol * np.sqrt(dt)) - (1 + d_1 * d_2) / (2 * dt) )
    return veta

def find_charm_call(s_t, k, r, r_f, vol, dt):
    d_1_c = (1 / (vol * (dt ** 0.5))) 
    d1_log =  (np.log((s_t)/(k)) + ((r + (vol ** 2) / 2) * dt))
    d_1 = d_1_c * d1_log
    d_2 = d_1 - vol * (dt ** 0.5)
    q = r_f - r
    charm = q * np.exp(-q*dt) * norm.cdf(d_1) - \
            np.exp(-q*dt) * norm.pdf(d_1) * \
            (( 2* (r_f - q) * dt - d_2 * vol * np.sqrt(dt)) / (2 * dt * vol * np.sqrt(dt)))
    return charm
 
def find_charm_put(s_t, k, r, r_f, vol, dt):
    d_1_c = (1 / (vol * (dt ** 0.5))) 
    d1_log =  (np.log((s_t)/(k)) + ((r + (vol ** 2) / 2) * dt))
    d_1 = d_1_c * d1_log
    d_2 = d_1 - vol * (dt ** 0.5)
    q = r_f - r
    charm = - q * np.exp(-q*dt) * norm.cdf(-d_1) - \
            np.exp(-q*dt) * norm.pdf(d_1) * \
            (( 2* (r_f - q) * dt - d_2 * vol * np.sqrt(dt)) / (2 * dt * vol * np.sqrt(dt)))
    return charm


In [57]:
#########################
#User-defined functions #
#########################

def generate_weekly_static_pfl_weights(no_of_paths, mthly_expiries_list, \
                                       dict_wkly_expiries_each_month, \
                                       dict_ce_wkly_strikes, dict_pe_wkly_strikes, \
                                       dict_wkly_spots, dict_wkly_iv, \
                                       prod_type, prod_moneyness, output_path, \
                                       option_type = "CE", no_of_assets=1, cor_mat=[[1]],\
                                       batch_size_divisor=10, no_of_epochs = 1000, stock_ident="BANKNIFTY"):
    ce_pfl_weights = []
    pe_pfl_weights = [] 
    cash_weights = []

    strike = 1000
    weekly_pfl_weights = []
    ce_dict = {}
    pe_dict = {}
    cash_dict = {}
    no_of_weeks = 1
        
    for week in range(0, no_of_weeks):
#         for week in range(0, 1):
            s_t = 1000
            K = strike          
            r = 0.04
            

            sim_start_date = date(2020,3,1)
            sim_end_date = date(2020, 2, 28)
            each_month = date(2021, 2, 28)
            
            option_moneyness = float(s_t/float(K))
            sim_moneyness = 1.0
            price_moneyness = option_moneyness
            
            vol_list = [0.2]
            
            sim_stock_mat = gbm1W_simulation(no_of_paths, sim_start_date, sim_end_date, \
                                             no_of_assets, s_t, r, vol_list, cor_mat)
            
            call_strikes = np.sort(np.array(call_strikes_list)).reshape(1,-1)
            put_strikes = np.sort(np.array(put_strikes_list)).reshape(1,-1)
            no_of_calls = call_strikes.size
            no_of_puts = put_strikes.size
            no_ind_vars = no_of_calls + no_of_puts
                        
            stock_vec = sim_stock_mat[:, 1]
            r_f = 0.04
            dt = float((each_month - sim_end_date).days) / 365

            # Need higher tenor in the first element to avoiding tenor_list[1] as missing when 2 tenors are not available
            
            vol = vol_list[0]                       
            opt_strike = strike

            if (option_type == "CE" ):
                option_value = BSM_call_vec_with_div(stock_vec, opt_strike, r, r_f, vol, dt)
            else:
                option_value = BSM_put_vec_with_div(stock_vec, opt_strike, r, r_f, vol, dt) 

    
            option_value = np.asarray(option_value)
            option_value = option_value.reshape(-1, 1) 
            
            y = option_value
            stock_vec = stock_vec.reshape(-1,1)
            x = np.concatenate((np.maximum(stock_vec-call_strikes,0), np.maximum(put_strikes-stock_vec,0)), axis=1) 
            
           
            
#             print("SKLearn Linear regression - OLS")
#             lin_model = LinearRegression().fit(x, y)
#             pfl_weights = lin_model.coef_
#             pfl_weights = pfl_weights.reshape(-1)
#             cash_weight = lin_model.intercept_
#             print("R-Square: ", lin_model.score(x,y))
#             print("Coefficients:")
#             print(pfl_weights)
#             print(cash_weight)           
            
#             print("SKLearn Linear regression - Ridge")
#             lin_model_ridge = Ridge(alpha=1.0).fit(x, y)
#             pfl_weights = lin_model_ridge.coef_
#             pfl_weights = pfl_weights.reshape(-1)
#             cash_weight = lin_model_ridge.intercept_
#             print("R-Square: ", lin_model_ridge.score(x, y))
#             print("Coefficients:")
#             print(pfl_weights)
#             print(cash_weight)
            
            
            print("SKLearn Linear regression - Lasso")
            lin_model_lasso = Lasso(alpha=1.0).fit(x, y)
            pfl_weights = lin_model_lasso.coef_
            pfl_weights = pfl_weights.reshape(-1)
            cash_weight = lin_model_lasso.intercept_
            print("R-Square: ", lin_model_lasso.score(x, y))
            print("Coefficients:")
            print(pfl_weights)
            print(cash_weight)
            

#             #Find Optimal Alpha
#             alphas = np.array([-0.5, 0.0, 0.25, 0.5, 1.0, 1.5, 2.0, 3.0, 4.0, 5.0])
#             model = LassoCV(alphas=alphas)
#             visualizer = AlphaSelection(model)
#             visualizer.fit(x, y)
#             visualizer.show()
            
#             print("Stats Model _ OLS")
#             x = x
#             x = sm.add_constant(x)
#             lin_model2 = OLS(y,x).fit()
#             print(lin_model2.summary())
# #             print(lin_model2.params)
# #           pfl_weights = np.array(lin_model2.params).reshape(-1)
            
            
            ######################################################################################
            ### Portfolio weights of shorter term options from neural network of European options ###
            ######################################################################################
            
            # Call Pfl weights
            call_strikes = call_strikes.reshape(-1) 
            ce_dates_list = np.array([sim_start_date.strftime("%d-%b-%Y") for i in range(0, no_of_calls)]).reshape(-1)
            ce_wt_list = pfl_weights[0:no_of_calls]
            ce_dict = {"Date": ce_dates_list, "Strike": call_strikes, "Pfl_weights": ce_wt_list}
            df_ce = pd.DataFrame(ce_dict)
            ce_pfl_weights.append(df_ce)
    
            
            #Put Pfl Weights
            put_strikes = put_strikes.reshape(-1) 
            pe_dates_list = np.array([sim_start_date.strftime("%d-%b-%Y") for i in range(0, no_of_puts)]).reshape(-1)
            pe_wt_list = pfl_weights[no_of_calls:no_ind_vars]
            pe_dict = {"Date": pe_dates_list, "Strike":put_strikes, "Pfl_weights": pe_wt_list}
            df_pe = pd.DataFrame(pe_dict)
            pe_pfl_weights.append(df_pe)
            
            # Cash component
            cash_date = [sim_start_date.strftime("%d-%b-%Y")]
            cash_dict = {"Date":cash_date, "Pfl_weights":cash_weight} 
            df_cash = pd.DataFrame(cash_dict)
            cash_weights.append(df_cash)
            
            
            df_ce_pfl_weights = pd.concat(ce_pfl_weights, axis=0)
            df_pe_pfl_weights = pd.concat(pe_pfl_weights, axis=0)
            df_cash_pfl_weights = pd.concat(cash_weights, axis=0)

            df_ce_pfl_weights.to_csv(out_PnL_path  + "F6_LR_" + stock_ident + "_" + prod_moneyness + "_" + prod_type + "_CE_Pfl_weights.csv", index = False)
            df_pe_pfl_weights.to_csv(out_PnL_path + "F6_LR_" + stock_ident + "_" + prod_moneyness + "_" + prod_type + "_PE_Pfl_weights.csv", index = False)
            df_cash_pfl_weights.to_csv(out_PnL_path  + "F6_LR_" + stock_ident + "_" + prod_moneyness + "_" + prod_type + "_Cash_Pfl_weights.csv", index = False)
    
    return df_ce_pfl_weights, df_pe_pfl_weights, df_cash_pfl_weights

# def output_strikes_hedged(prod_type, prod_moneyness, dict_mthly_strikes, stock_ident, output_path):
#     list_of_dates = []
#     list_of_prod_type = []
#     list_of_strikes = []
#     for each_month in dict_mthly_strikes.keys():
#         list_of_dates.append(each_month)
#         list_of_strikes.append(dict_mthly_strikes[each_month])
#         list_of_prod_type.append(prod_type)
#     dict_hedged_prod = {"Date":list_of_dates, "prod_type":list_of_prod_type, "Strike":list_of_strikes}
#     df_hedged_prod = pd.DataFrame(dict_hedged_prod) 
# #     df_hedged_prod.to_csv(output_path + "B1A_LR_" + stock_ident + "_" + prod_moneyness + "_" + prod_type + "_strikes_hedged.csv", index = False)
#     return(df_hedged_prod)


In [61]:

#Find the monthly strikes of option from mkt data to find the option to be hedged
#Every month, an option is hedged
mthly_expiries_list = get_all_monthly_option_expiries(periods_of_interest, holidays_list)
dict_wkly_expiries_each_month = find_wkly_expries(mthly_expiries_list, holidays_list)

# #Load all monthly mkt data
mthly_mkt_data = load_all_mthly_data(mthly_expiries_list, input_data_path, holidays_list, prod_type_lists=["FUT", "CE", "PE"], stock_ident=stock_ident)

# #Load weekly strikes of short term options used for hedging monthly options 
dict_ce_wkly_strikes, dict_pe_wkly_strikes, dict_wkly_spots = generate_weekly_strikes(dict_wkly_expiries_each_month, input_data_path, stock_ident = stock_ident)     

#Load monthly strikes of interest as of required moneyness on the begining of month
dict_mthly_strikes = generate_mthly_strikes(mthly_mkt_data, prod_moneyness, prod_type, holidays_list, stock_ident=stock_ident)

#Load weekly IV for Black-Scholes model input and monte-carlo simulation of stock paths
dict_wkly_iv = generate_weekly_iv(dict_wkly_expiries_each_month, dict_wkly_spots, dict_mthly_strikes, prod_type, output_path=output_data_path, atm_ind=0, stock_ident=stock_ident)

#Load weekly risk free interest rates from futures
#This function to be changed based on updates on risk-free-interest - now kept it as cosntant
dict_wkly_ir = generate_weekly_ir(dict_wkly_expiries_each_month, mthly_mkt_data, stock_ident)

# # #Load Static portfolio weights built by neural network
ce_pfl_wts, pe_pfl_wts, cash_pfl_wts = generate_weekly_static_pfl_weights(no_of_paths, mthly_expiries_list, dict_wkly_expiries_each_month, \
                                                        dict_ce_wkly_strikes, dict_pe_wkly_strikes, dict_wkly_spots, \
                                                        dict_wkly_iv, prod_type, prod_moneyness, output_data_path, \
                                                        option_type = prod_type, no_of_assets=no_of_assets, \
                                                        cor_mat=cor_mat, stock_ident=stock_ident) 

# # # The strike correspnding to the option that is hedged
# df_hedged_prod = output_strikes_hedged(prod_type, prod_moneyness, dict_mthly_strikes, stock_ident, output_data_path)



SKLearn Linear regression - Lasso
R-Square:  0.9996678445844597
Coefficients:
[ 0.63942891  0.          0.          0.          0.         -0.57788083]
[99.25495672]


ipynb.fs.full.user_defined_vik_functions:1173: RuntimeWarning: covariance is not positive-semidefinite.


In [63]:
###########################################################
# Target Option PnL Attribution - 7th April - NIFTY CALL ##
###########################################################

prv_day = date(2020,3,5)
nxt_day = date(2020,3,6)
begin_wk = date(2020,3,1)
end_wk = date(2020,3,31)
end_mth = date(2021,2,28)
prev_price_S0 = 1000 
next_price_S1 = 1000
K=1000
dt0 = ((end_mth - prv_day).days)/365
dt1 = ((end_mth - nxt_day).days)/365

dtshort0 = ((end_wk - prv_day).days)/365
dtshort1 = ((end_wk - nxt_day).days)/365

iv0 = 0.2
iv1 = 0.21
ivshort0 = 0.2
ivshort1 = 0.21

MV0 = BSM_call_vec_with_div(prev_price_S0, K, 0.04, 0.04, iv0,  dt0)
MV1 = BSM_call_vec_with_div(next_price_S1, K, 0.04, 0.04, iv1,  dt1)
print(MV0)
summary_dict = {"Option Type": ["CE"], "Moneyness": ["ATM"], \
                "S0": [prev_price_S0 for i in range(0,1)], "S1": [next_price_S1 for i in range(0,1)], "r_f": [0.04 for i in range(0,1)], \
#                  "r": [0.00117056 for i in range(0,3)], "q": [0.03882944 for i in range(0,3)], \
                 "r": [0.04 for i in range(0,1)], "q": [0.0 for i in range(0,1)], \
                 "K": [1000], "MV0": [MV0], "MV1": [MV1], \
                  "dt0": [((end_mth - prv_day).days)/365 for i in range(0,1)], \
                  "dt1": [((end_mth - nxt_day).days)/365 for i in range(0,1)]}

#Incremental Analysis

#Long Term Option
df_summary = pd.DataFrame(summary_dict)

df_summary["iv0"] = df_summary.apply(lambda x: iv0, axis=1 )
df_summary["iv1"] = df_summary.apply(lambda x: iv1, axis=1 )

df_summary["BS0"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["BS1"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"]), axis=1 )

df_summary["BS0_ds"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["BS0_dsdv"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"]), axis=1 )
df_summary["BS0_dsdt"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )

df_summary["BS0_dv"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"]), axis=1 )
df_summary["BS0_dvds"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"]), axis=1 )
df_summary["BS0_dvdt"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"]), axis=1 )

df_summary["BS0_dt"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
df_summary["BS0_dtds"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
df_summary["BS0_dtdv"] = df_summary.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"]), axis=1 )


# Greeks
df_summary["delta0"] = df_summary.apply(lambda x: call_delta_with_spot_and_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["gamma0"] = df_summary.apply(lambda x: find_gamma(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["vega0"] = df_summary.apply(lambda x: find_vega(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["volga0"] = df_summary.apply(lambda x: find_volga(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["vanna0"] = df_summary.apply(lambda x: find_vanna(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["theta0"] = df_summary.apply(lambda x: find_theta_Call(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["veta0"] = df_summary.apply(lambda x: find_veta(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["speed0"] = df_summary.apply(lambda x: find_speed(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_summary["charm0"] = df_summary.apply(lambda x: find_charm_call(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )

# df_summary["delta0"] = df_summary.apply(lambda x: call_delta_with_spot_and_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_summary["gamma0"] = df_summary.apply(lambda x: find_gamma(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_summary["vega0"] = df_summary.apply(lambda x: find_vega(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_summary["volga0"] = df_summary.apply(lambda x: find_volga(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_summary["vanna0"] = df_summary.apply(lambda x: find_vanna(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_summary["theta0"] = df_summary.apply(lambda x: find_theta_Call(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_summary["veta0"] = df_summary.apply(lambda x: find_veta(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_summary["speed0"] = df_summary.apply(lambda x: find_speed(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )
# df_summary["charm0"] = df_summary.apply(lambda x: find_charm_call(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )


#PnL
df_summary["delta PnL"] = df_summary.apply(lambda x: x["delta0"] * (x["S1"] - x["S0"]), axis=1 )
df_summary["gamma PnL"] = df_summary.apply(lambda x: 0.5 * x["gamma0"] * (np.square(x["S1"] - x["S0"])), axis=1 )
df_summary["vega PnL"] = df_summary.apply(lambda x: x["vega0"] * (x["iv1"] - x["iv0"]), axis=1 )
df_summary["volga PnL"] = df_summary.apply(lambda x: 0.5 * x["volga0"] * ((x["iv1"] - x["iv0"])**2), axis=1 )
df_summary["vanna PnL"] = df_summary.apply(lambda x: x["vanna0"] * (x["iv1"] - x["iv0"]) *(x["S1"] - x["S0"]), axis=1 )
df_summary["theta PnL"] = df_summary.apply(lambda x: x["theta0"] * (x["dt0"] - x["dt1"]) , axis=1 )
df_summary["veta PnL"] = df_summary.apply(lambda x: x["veta0"] * (x["iv1"] - x["iv0"]) * (x["dt0"] - x["dt1"]), axis=1 )
df_summary["speed PnL"] = df_summary.apply(lambda x: (1/6.0) * x["speed0"] * (np.power((x["S1"] - x["S0"]),3)), axis=1 )
df_summary["charm PnL"] = df_summary.apply(lambda x: x["charm0"] * (x["S1"] - x["S0"]) * (x["dt0"] - x["dt1"]), axis=1 )

                                         
df_summary["Target Option Realised PnL"] = df_summary["MV1"] - df_summary["MV0"]
df_summary["BS PnL"] = df_summary["BS1"] - df_summary["BS0"]


#Marginal PnLs
df_summary["BS ds PnL"] = df_summary["BS0_ds"] - df_summary["BS0"]
df_summary["BS dv PnL"] = df_summary["BS0_dv"] - df_summary["BS0"]
df_summary["BS dt PnL"] = df_summary["BS0_dt"] - df_summary["BS0"]

df_summary["BS dsdv PnL"] = df_summary["BS0_dsdv"] - df_summary["BS0_ds"]
df_summary["BS dsdt PnL"] = df_summary["BS0_dsdt"] - df_summary["BS0_ds"]

df_summary["BS dvds PnL"] = df_summary["BS0_dvds"] - df_summary["BS0_dv"]
df_summary["BS dvdt PnL"] = df_summary["BS0_dvdt"] - df_summary["BS0_dv"]

df_summary["BS dtds PnL"] = df_summary["BS0_dtds"] - df_summary["BS0_dt"]
df_summary["BS dtdv PnL"] = df_summary["BS0_dtdv"] - df_summary["BS0_dt"]


df_summary["BS dsdvdt PnL"] = df_summary["BS1"] - df_summary["BS0_dsdv"]
df_summary["BS dsdtdv PnL"] = df_summary["BS1"] - df_summary["BS0_dsdt"]
df_summary["BS dvdsdt PnL"] = df_summary["BS1"] - df_summary["BS0_dvds"]
df_summary["BS dvdtds PnL"] = df_summary["BS1"] - df_summary["BS0_dvdt"]
df_summary["BS dtdsdv PnL"] = df_summary["BS1"] - df_summary["BS0_dtds"]
df_summary["BS dtdvds PnL"] = df_summary["BS1"] - df_summary["BS0_dtdv"]


df_summary["Dynamic Hedge PnL"] = df_summary.apply(lambda x: (x["delta0"] * x["S1"] + \
                                                   (x["MV0"] - x["delta0"] * x["S0"]) * np.exp( summary_dict["r_f"][0] *  ((date(2020,4,7) - date(2020, 4, 3)).days)/365 ) - \
                                                   x["MV0"]), axis=1 )

df_summary.to_csv(out_PnL_path + "F7_pnl_TEST_targetOption_" + nxt_day.strftime("%d-%b-%Y") + ".csv")
# df_summary.to_csv(out_PnL_path + "pnl_attribution_targetOption_" + nxt_day.strftime("%d-%b-%Y") + "day1.csv")


#Short Term Option Hedge Pfl

#Market Value

call_strikes = np.sort(np.array(call_strikes_list)).reshape(-1)
put_strikes = np.sort(np.array(put_strikes_list)).reshape(-1)
call0 = BSM_call_vec_with_div(prev_price_S0, call_strikes, 0.04, 0.04, ivshort0, dtshort0)
call1 = BSM_call_vec_with_div(next_price_S1, call_strikes, 0.04, 0.04, ivshort1, dtshort1)
put0 = BSM_put_vec_with_div(prev_price_S0, put_strikes, 0.04, 0.04, ivshort0, dtshort0)
put1 = BSM_put_vec_with_div(next_price_S1, put_strikes, 0.04, 0.04, ivshort1, dtshort1)
df_ce_mv0 = pd.DataFrame({"Strike Price":call_strikes , "Settle Price":call0 })
df_ce_mv1 = pd.DataFrame({"Strike Price":call_strikes , "Settle Price":call1 })
df_pe_mv0 = pd.DataFrame({"Strike Price":put_strikes , "Settle Price":put0 })
df_pe_mv1 = pd.DataFrame({"Strike Price":put_strikes , "Settle Price":put1 })
print(df_ce_mv0)
print(df_ce_mv1)
print(df_pe_mv0)
print(df_pe_mv1)
# moneyness_list = ["ATM", "ITM", "OTM"]
moneyness_list = ["ATM"]

df_mon_list = []
for moneyness in moneyness_list:
    df_ce_pfl_wts = pd.read_csv(out_PnL_path + "F6_LR_" + stock_ident + "_" + prod_moneyness + "_" + prod_type + "_CE_Pfl_weights.csv")
    df_pe_pfl_wts = pd.read_csv(out_PnL_path + "F6_LR_" + stock_ident + "_" + prod_moneyness + "_" + prod_type + "_PE_Pfl_weights.csv")    
    df_cash_pfl_wts = pd.read_csv(out_PnL_path + "F6_LR_" + stock_ident + "_" + prod_moneyness + "_" + prod_type + "_Cash_Pfl_weights.csv")
    cash0 = df_cash_pfl_wts[df_cash_pfl_wts["Date"] == begin_wk.strftime("%d-%b-%Y")]["Pfl_weights"].values[0] * np.exp(-summary_dict["r_f"][0] * ((end_wk - prv_day).days)/365)
    cash1 = df_cash_pfl_wts[df_cash_pfl_wts["Date"] == begin_wk.strftime("%d-%b-%Y")]["Pfl_weights"].values[0] * np.exp(-summary_dict["r_f"][0] * ((end_wk - nxt_day).days)/365)

    #Calls
    df_ce_pfl_wts = df_ce_pfl_wts[df_ce_pfl_wts["Date"] == (begin_wk).strftime("%d-%b-%Y")]
    df_ce_pfl_wts = df_ce_pfl_wts[df_ce_pfl_wts["Pfl_weights"] != 0]
    l1 = len(df_ce_pfl_wts.axes[0])
    df_ce_pfl_wts["Option Type"] = ["CE" for i in range(0,l1)]
    df_ce_pfl_wts = df_ce_pfl_wts.merge(df_ce_mv0, how="left", left_on=["Strike"], right_on=["Strike Price"])
    df_ce_pfl_wts = df_ce_pfl_wts.rename(columns={"Settle Price": 'MV0'})
    df_ce_pfl_wts = df_ce_pfl_wts.merge(df_ce_mv1, how="left", left_on=["Strike"], right_on=["Strike Price"])
    df_ce_pfl_wts = df_ce_pfl_wts.rename(columns={"Settle Price": 'MV1'})
    df_ce_pfl_wts = df_ce_pfl_wts.rename(columns={"Strike": "K"})
    df_ce_pfl_wts = df_ce_pfl_wts[["Option Type", "K", "Pfl_weights", "MV0", "MV1"]]

    #Puts
    df_pe_pfl_wts = df_pe_pfl_wts[df_pe_pfl_wts["Date"] == (begin_wk).strftime("%d-%b-%Y")]
    df_pe_pfl_wts = df_pe_pfl_wts[df_pe_pfl_wts["Pfl_weights"] != 0]    
    l2 = len(df_pe_pfl_wts.axes[0])
    df_pe_pfl_wts["Option Type"] = ["PE" for i in range(0,l2)]
    df_pe_pfl_wts = df_pe_pfl_wts.merge(df_pe_mv0, how="left", left_on=["Strike"], right_on=["Strike Price"])
    df_pe_pfl_wts = df_pe_pfl_wts.rename(columns={"Settle Price": 'MV0'})
    df_pe_pfl_wts = df_pe_pfl_wts.merge(df_pe_mv1, how="left", left_on=["Strike"], right_on=["Strike Price"])
    df_pe_pfl_wts = df_pe_pfl_wts.rename(columns={"Settle Price": 'MV1'})
    df_pe_pfl_wts = df_pe_pfl_wts.rename(columns={"Strike": "K"})
    df_pe_pfl_wts = df_pe_pfl_wts[["Option Type", "K", "Pfl_weights", "MV0", "MV1"]]
    
    df_wts_opt = df_ce_pfl_wts.append(df_pe_pfl_wts, ignore_index = True)
    
    
    l2 = len(df_wts_opt.axes[0])
    df_wts_opt["Moneyness"] = [moneyness for i in range(0,l2)]
    df_wts_opt["cash0"] = [cash0 for i in range(0,l2)]
    df_wts_opt["cash1"] = [cash1 for i in range(0,l2)]
    df_mon_list.append(df_wts_opt)

df_wts = pd.concat(df_mon_list)
l = len(df_wts.axes[0])
S0 = [summary_dict["S0"][0] for i in range(0,l)]
S1 = [summary_dict["S1"][0] for i in range(0,l)]

r = [0.04 for i in range(0,l)]
r_f = [0.04 for i in range(0,l)]
q = [0 for i in range(0,l)]
df_wts["S0"] = S0
df_wts["S1"] = S1
df_wts["dt0"] = [((end_wk - prv_day).days)/365 for i in range(0,l)]
df_wts["dt1"] = [((end_wk - nxt_day).days)/365 for i in range(0,l)]
df_wts["r"] = r
df_wts["r_f"] = r_f
df_wts["q"] = q
df_hdg_pfl = df_wts

df_hdg_pfl["iv0"] = df_hdg_pfl.apply(lambda x: ivshort0, axis=1 )
df_hdg_pfl["iv1"] = df_hdg_pfl.apply(lambda x: ivshort1, axis=1 )

df_hdg_pfl["BS0"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"], x["dt0"]) if x["Option Type"] == "CE" \
                                    else BSM_put_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_hdg_pfl["BS1"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"]), axis=1 )


df_hdg_pfl["BS0_ds"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]) , axis=1 )
df_hdg_pfl["BS0_dsdv"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"]) , axis=1 )
df_hdg_pfl["BS0_dsdt"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]) , axis=1 )


df_hdg_pfl["BS0_dv"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"]) , axis=1 )
df_hdg_pfl["BS0_dvds"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt0"]) , axis=1 )
df_hdg_pfl["BS0_dvdt"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"]) , axis=1 )


df_hdg_pfl["BS0_dt"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"], x["dt1"]) if x["Option Type"] == "CE" \
                                    else BSM_put_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]), axis=1 )

df_hdg_pfl["BS0_dtds"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S1"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt1"]) , axis=1 )

df_hdg_pfl["BS0_dtdv"] = df_hdg_pfl.apply(lambda x: BSM_call_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"])  if x["Option Type"] == "CE" \
                                            else BSM_put_vec_with_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv1"],  x["dt1"]) , axis=1 )

df_hdg_pfl["wMV0"] = df_hdg_pfl.apply(lambda x: x["MV0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wMV1"] = df_hdg_pfl.apply(lambda x: x["MV1"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wBS0"] = df_hdg_pfl.apply(lambda x: x["BS0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wBS1"] = df_hdg_pfl.apply(lambda x: x["BS1"] * x["Pfl_weights"], axis=1 )

df_hdg_pfl["wBS0_ds"] = df_hdg_pfl.apply(lambda x: x["BS0_ds"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wBS0_dv"] = df_hdg_pfl.apply(lambda x: x["BS0_dv"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wBS0_dt"] = df_hdg_pfl.apply(lambda x: x["BS0_dt"] * x["Pfl_weights"], axis=1 )

df_hdg_pfl["wBS0_dsdv"] = df_hdg_pfl.apply(lambda x: x["BS0_dsdv"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wBS0_dsdt"] = df_hdg_pfl.apply(lambda x: x["BS0_dsdt"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wBS0_dvds"] = df_hdg_pfl.apply(lambda x: x["BS0_dvds"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wBS0_dvdt"] = df_hdg_pfl.apply(lambda x: x["BS0_dvdt"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wBS0_dtds"] = df_hdg_pfl.apply(lambda x: x["BS0_dtds"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wBS0_dtdv"] = df_hdg_pfl.apply(lambda x: x["BS0_dtdv"] * x["Pfl_weights"], axis=1 )


df_hdg_pfl["delta0"] = df_hdg_pfl.apply(lambda x: call_delta_with_spot_and_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]) if x["Option Type"] == "CE" \
                                            else put_delta_with_spot_and_div(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_hdg_pfl["gamma0"] = df_hdg_pfl.apply(lambda x: find_gamma(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_hdg_pfl["vega0"] = df_hdg_pfl.apply(lambda x: find_vega(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_hdg_pfl["volga0"] = df_hdg_pfl.apply(lambda x: find_volga(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_hdg_pfl["vanna0"] = df_hdg_pfl.apply(lambda x: find_vanna(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_hdg_pfl["theta0"] = df_hdg_pfl.apply(lambda x: find_theta_Call(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]) if x["Option Type"] == "CE" \
                                            else find_theta_Put(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_hdg_pfl["veta0"] = df_hdg_pfl.apply(lambda x: find_veta(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_hdg_pfl["speed0"] = df_hdg_pfl.apply(lambda x: find_speed(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )
df_hdg_pfl["charm0"] = df_hdg_pfl.apply(lambda x: find_charm_call(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]) if x["Option Type"] == "CE" \
                                            else find_charm_put(x["S0"], x["K"], x["r"], x["r_f"], x["iv0"],  x["dt0"]), axis=1 )



df_hdg_pfl["wdelta0"] = df_hdg_pfl.apply(lambda x: x["delta0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wgamma0"] = df_hdg_pfl.apply(lambda x: x["gamma0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wvega0"] = df_hdg_pfl.apply(lambda x: x["vega0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wvolga0"] = df_hdg_pfl.apply(lambda x: x["volga0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wvanna0"] = df_hdg_pfl.apply(lambda x: x["vanna0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wtheta0"] = df_hdg_pfl.apply(lambda x: x["theta0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wveta0"] = df_hdg_pfl.apply(lambda x: x["veta0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wspeed0"] = df_hdg_pfl.apply(lambda x: x["speed0"] * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wcharm0"] = df_hdg_pfl.apply(lambda x: x["charm0"] * x["Pfl_weights"], axis=1 )

df_hdg_pfl["Hdg Pfl Realised PnL"] = df_hdg_pfl.apply(lambda x: (x["MV1"] - x["MV0"]) * x["Pfl_weights"] + (x["cash1"] - x["cash0"]), axis=1 )
df_hdg_pfl["Hdg Pfl PnL"] = df_hdg_pfl.apply(lambda x: (x["BS1"] - x["BS0"]) * x["Pfl_weights"] + (x["cash1"] - x["cash0"]), axis=1 )
df_hdg_pfl["Hdg Pfl Realised Options PnL"] = df_hdg_pfl.apply(lambda x: (x["MV1"] - x["MV0"]) * x["Pfl_weights"], axis=1 )
df_hdg_pfl["wdelta PnL"] = df_hdg_pfl.apply(lambda x: x["wdelta0"] * (x["S1"] - x["S0"]), axis=1 )
df_hdg_pfl["wgamma PnL"] = df_hdg_pfl.apply(lambda x: 0.5 * x["wgamma0"] * (np.square(x["S1"] - x["S0"])), axis=1 )
df_hdg_pfl["wvega PnL"] = df_hdg_pfl.apply(lambda x: x["wvega0"] * (x["iv1"] - x["iv0"]), axis=1 )
df_hdg_pfl["wvolga PnL"] = df_hdg_pfl.apply(lambda x: 0.5* x["wvolga0"] * np.square(x["iv1"] - x["iv0"]), axis=1 )
df_hdg_pfl["wvanna PnL"] = df_hdg_pfl.apply(lambda x: x["wvanna0"] * (x["iv1"] - x["iv0"]) * (x["S1"] - x["S0"]), axis=1 )
df_hdg_pfl["wtheta PnL"] = df_hdg_pfl.apply(lambda x: x["wtheta0"] * (x["dt0"] - x["dt1"]) + (x["cash1"] - x["cash0"]), axis=1 )                                        
df_hdg_pfl["wveta PnL"] = df_hdg_pfl.apply(lambda x: x["wveta0"] * (x["iv1"] - x["iv0"]) * (x["dt0"] - x["dt1"]), axis=1 )
df_hdg_pfl["wspeed PnL"] = df_hdg_pfl.apply(lambda x: (1/6.0) * x["wspeed0"] * np.power((x["S1"] - x["S0"]), 3), axis=1 )
df_hdg_pfl["wcharm PnL"] = df_hdg_pfl.apply(lambda x: x["wcharm0"] * (x["S1"] - x["S0"]) * (x["dt0"] - x["dt1"]), axis=1 )


# print(df_hdg_pfl)

hdge_cash0 = []
hdge_cash1 = []

hdg_MV0 = []
hdge_MV1 = []
hdge_BS0 = []
hdge_BS1 = []

hdge_BS0_ds = []
hdge_BS0_dsdv = []
hdge_BS0_dsdt = []

hdge_BS0_dv = []
hdge_BS0_dvds = []
hdge_BS0_dvdt = []

hdge_BS0_dt = []
hdge_BS0_dtds = []
hdge_BS0_dtdv = []

hdge_delta0 = []
hdge_gamma0 = []
hdge_vega0 = []
hdge_volga0 = []
hdge_vanna0 = []
hdge_theta0 = []
hdge_veta0 = []
hdge_speed0 = []
hdge_charm0 = []

hdgPflrealPnL = []
hdgPflrealOptionsPnL = []
hdgPflPnL = []
hdge_deltaPnL = []
hdge_gammaPnL = []
hdge_vegaPnL = []
hdge_volgaPnL = []
hdge_vannaPnL = []
hdge_thetaPnL = []
hdge_vetaPnL = []
hdge_speedPnL = []
hdge_charmPnL = []


for moneyness in moneyness_list:
    df_hdg_sub_pfl = df_hdg_pfl[df_hdg_pfl["Moneyness"] == moneyness]
    
    hdge_cash0.append(df_hdg_sub_pfl["cash0"].values[0])
    hdge_cash1.append(df_hdg_sub_pfl["cash1"].values[0])     
    hdg_MV0.append(np.sum(df_hdg_sub_pfl["wMV0"]))
    hdge_MV1.append(np.sum(df_hdg_sub_pfl["wMV1"]))
    
    hdge_BS0.append(np.sum(df_hdg_sub_pfl["wBS0"]))
    hdge_BS1.append(np.sum(df_hdg_sub_pfl["wBS1"]))    
    
    hdge_BS0_ds.append(np.sum(df_hdg_sub_pfl["wBS0_ds"]))
    hdge_BS0_dsdv.append(np.sum(df_hdg_sub_pfl["wBS0_dsdv"]))
    hdge_BS0_dsdt.append(np.sum(df_hdg_sub_pfl["wBS0_dsdt"]))

    hdge_BS0_dv.append(np.sum(df_hdg_sub_pfl["wBS0_dv"]))
    hdge_BS0_dvds.append(np.sum(df_hdg_sub_pfl["wBS0_dvds"]))
    hdge_BS0_dvdt.append(np.sum(df_hdg_sub_pfl["wBS0_dvdt"]))

    hdge_BS0_dt.append(np.sum(df_hdg_sub_pfl["wBS0_dt"]))
    hdge_BS0_dtds.append(np.sum(df_hdg_sub_pfl["wBS0_dtds"]))
    hdge_BS0_dtdv.append(np.sum(df_hdg_sub_pfl["wBS0_dtdv"]))

    hdge_delta0.append(np.sum(df_hdg_sub_pfl["wdelta0"]))
    hdge_gamma0.append(np.sum(df_hdg_sub_pfl["wgamma0"]))
    hdge_vega0.append(np.sum(df_hdg_sub_pfl["wvega0"]))
    hdge_volga0.append(np.sum(df_hdg_sub_pfl["wvolga0"]))
    hdge_vanna0.append(np.sum(df_hdg_sub_pfl["wvanna0"]))
    hdge_theta0.append(np.sum(df_hdg_sub_pfl["wtheta0"]))
    hdge_veta0.append(np.sum(df_hdg_sub_pfl["wveta0"]))
    hdge_speed0.append(np.sum(df_hdg_sub_pfl["wspeed0"]))
    hdge_charm0.append(np.sum(df_hdg_sub_pfl["wcharm0"]))
   
    hdge_deltaPnL.append(np.sum(df_hdg_sub_pfl["wdelta PnL"]))
    hdge_gammaPnL.append(np.sum(df_hdg_sub_pfl["wgamma PnL"]))
    hdge_vegaPnL.append(np.sum(df_hdg_sub_pfl["wvega PnL"]))
    hdge_volgaPnL.append(np.sum(df_hdg_sub_pfl["wvolga PnL"]))
    hdge_vannaPnL.append(np.sum(df_hdg_sub_pfl["wvanna PnL"]))
    hdge_thetaPnL.append(np.sum(df_hdg_sub_pfl["wtheta PnL"]))
    hdge_vetaPnL.append(np.sum(df_hdg_sub_pfl["wveta PnL"]))
    hdge_speedPnL.append(np.sum(df_hdg_sub_pfl["wspeed PnL"]))
    hdge_charmPnL.append(np.sum(df_hdg_sub_pfl["wcharm PnL"]))

    hdgPflPnL.append(np.sum(df_hdg_sub_pfl["Hdg Pfl PnL"]))
    hdgPflrealPnL.append(np.sum(df_hdg_sub_pfl["Hdg Pfl Realised PnL"]))
    hdgPflrealOptionsPnL.append(np.sum(df_hdg_sub_pfl["Hdg Pfl Realised Options PnL"]))
    
dict_summary_hdg = {"cash0":hdge_cash0, "cash1": hdge_cash1, "Moneyness": moneyness_list, "MV0": hdg_MV0, "MV1": hdge_MV1, \
                     "BS0":hdge_BS0, "BS1":hdge_BS1, \
                    "BS0_ds":hdge_BS0_ds, "BS0_dsdv":hdge_BS0_dsdv, "BS0_dsdt":hdge_BS0_dsdt, \
                    "BS0_dv":hdge_BS0_dv, "BS0_dvds":hdge_BS0_dvds, "BS0_dvdt":hdge_BS0_dvdt, \
                    "BS0_dt":hdge_BS0_dt, "BS0_dtds":hdge_BS0_dtds, "BS0_dtdv":hdge_BS0_dtdv, \
                    "delta0":hdge_delta0, "gamma0":hdge_gamma0, "vega0": hdge_vega0, \
                    "volga0":hdge_volga0, "vanna0":hdge_vanna0, "theta0":hdge_theta0, \
                    "veta0":hdge_veta0,  "speed0":hdge_speed0, "charm0":hdge_charm0, \
                    "delta PnL":hdge_deltaPnL, "gamma PnL":hdge_gammaPnL, "vega PnL": hdge_vegaPnL, \
                    "volga PnL":hdge_volgaPnL, "vanna PnL":hdge_vannaPnL, "theta PnL":hdge_thetaPnL, \
                    "veta PnL":hdge_vetaPnL,  "speed PnL":hdge_speedPnL, "charm PnL":hdge_charmPnL, \
                    "Hdge Pfl Realised PnL":hdgPflrealPnL, "Hdge Pfl Realised Options PnL": hdgPflrealOptionsPnL, "Hdge Pfl PnL":hdgPflPnL}


df_summary_hdg =pd.DataFrame(dict_summary_hdg)

df_summary_hdg["BS Options PnL"] = df_summary_hdg["BS1"] - df_summary_hdg["BS0"]

df_summary_hdg["BS ds PnL"] = df_summary_hdg["BS0_ds"] - df_summary_hdg["BS0"]
df_summary_hdg["BS dv PnL"] = df_summary_hdg["BS0_dv"] - df_summary_hdg["BS0"]
df_summary_hdg["BS dt PnL"] = df_summary_hdg["BS0_dt"] - df_summary_hdg["BS0"]

df_summary_hdg["BS dsdv PnL"] = df_summary_hdg["BS0_dsdv"] - df_summary_hdg["BS0_ds"]
df_summary_hdg["BS dsdt PnL"] = df_summary_hdg["BS0_dsdt"] - df_summary_hdg["BS0_ds"]

df_summary_hdg["BS dvds PnL"] = df_summary_hdg["BS0_dvds"] - df_summary_hdg["BS0_dv"]
df_summary_hdg["BS dvdt PnL"] = df_summary_hdg["BS0_dvdt"] - df_summary_hdg["BS0_dv"]

df_summary_hdg["BS dtds PnL"] = df_summary_hdg["BS0_dtds"] - df_summary_hdg["BS0_dt"]
df_summary_hdg["BS dtdv PnL"] = df_summary_hdg["BS0_dtdv"] - df_summary_hdg["BS0_dt"]


df_summary_hdg["BS dsdvdt PnL"] = df_summary_hdg["BS1"] - df_summary_hdg["BS0_dsdv"]
df_summary_hdg["BS dsdtdv PnL"] = df_summary_hdg["BS1"] - df_summary_hdg["BS0_dsdt"]
df_summary_hdg["BS dvdsdt PnL"] = df_summary_hdg["BS1"] - df_summary_hdg["BS0_dvds"]
df_summary_hdg["BS dvdtds PnL"] = df_summary_hdg["BS1"] - df_summary_hdg["BS0_dvdt"]
df_summary_hdg["BS dtdsdv PnL"] = df_summary_hdg["BS1"] - df_summary_hdg["BS0_dtds"]
df_summary_hdg["BS dtdvds PnL"] = df_summary_hdg["BS1"] - df_summary_hdg["BS0_dtdv"]



df_hdg_pfl.to_csv(out_PnL_path + "F7_pnl_TEST_hdgePfL_inter_" + nxt_day.strftime("%d-%b-%Y") + ".csv")
df_summary_hdg.to_csv(out_PnL_path + "F7_pnl_TEST_hedgePfl_" + nxt_day.strftime("%d-%b-%Y") + ".csv")



98.44192087609372
   Strike Price  Settle Price
0          1000  2.271522e+01
1          1200  5.854970e-03
2          1400  1.974682e-09
   Strike Price  Settle Price
0          1000  2.328810e+01
1          1200  8.649239e-03
2          1400  6.414584e-09
   Strike Price  Settle Price
0           600  1.340824e-21
1           800  1.178452e-04
2          1000  1.986996e+01
   Strike Price  Settle Price
0           600  2.055215e-20
1           800  2.141460e-04
2          1000  2.055212e+01
